In [1]:
# === CNT Quickstart Notebook — Single Cell (CPU/GPU autodetect) ===
# Purpose: one-cell bootstrap for a reproducible Kuramoto sweep with artifacts.
# Outputs:
#   - artifacts/<RUN_ID>/kuramoto_sweep.png
#   - artifacts/<RUN_ID>/kuramoto_sweep.csv
#   - artifacts/<RUN_ID>/run_meta.json

import os, json, math, time, sys, platform, uuid, datetime as dt
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# 0) Environment & paths
# ------------------------------------------------------------
# Preferred root (customize to your lab); falls back to local folder if unavailable
CANDIDATES = [Path("E:/CNT"), Path("E:\\CNT"), Path("./CNT")]
ROOT = next((p for p in CANDIDATES if p.exists()), Path(".").resolve())
ART = (ROOT / "artifacts").resolve()
RUN_ID = dt.datetime.utcnow().strftime("%Y%m%d-%H%M%SZ") + "-" + uuid.uuid4().hex[:6]
RUN_DIR = (ART / f"cnt_quickstart/{RUN_ID}")
RUN_DIR.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------
# 1) Backend selection (CuPy if available, else NumPy)
# ------------------------------------------------------------
try:
    import cupy as cp  # type: ignore
    # Sanity check on device availability
    _ = cp.zeros((1,))
    xp = cp
    HAS_CP = True
except Exception:
    xp = np
    HAS_CP = False

def to_np(a):
    """Safely convert xp-array to numpy array for saving/plotting."""
    if HAS_CP:
        return cp.asnumpy(a)
    return a

# ------------------------------------------------------------
# 2) Kuramoto simulator (vectorized; supports xp = np/cp)
# ------------------------------------------------------------
def kuramoto_R_tail(omega_xp, K, dt=0.01, steps=6000, tail_frac=0.5, seed=0):
    """
    Integrate Kuramoto phases: dθ_i/dt = ω_i + (K/N) * sum_j sin(θ_j - θ_i)
    Returns time-averaged order parameter R over the final tail of the simulation.
    """
    rs = xp.random.RandomState(seed) if not HAS_CP else xp.random.RandomState(seed)
    N = omega_xp.shape[0]
    theta = (2 * xp.pi) * rs.rand(N)
    steps = int(steps)
    tail_start = int((1.0 - tail_frac) * steps)

    # Pre-alloc for running average of R in tail
    R_sum, R_cnt = 0.0, 0

    for t in range(steps):
        # Compute the mean field via complex order parameter
        z = xp.exp(1j * theta)
        r = xp.abs(z.mean())  # instantaneous order parameter R_t

        if t >= tail_start:
            R_sum += float(r)
            R_cnt += 1

        # Mean-field coupling term: Im(e^{-iθ} * z_mean)
        z_mean = z.mean()
        coupling = xp.imag(z_mean * xp.exp(-1j * theta))
        theta = theta + dt * (omega_xp + K * coupling)
        # Keep phases numerically stable
        if t % 200 == 0:
            theta = xp.mod(theta, 2 * xp.pi)

    R_tail = R_sum / max(R_cnt, 1)
    return R_tail

# ------------------------------------------------------------
# 3) Experiment spec
# ------------------------------------------------------------
N = 1024                 # oscillators
SIGMA = 1.0              # freq std
DIST = "gaussian"        # 'gaussian' or 'lorentz'
K_MIN, K_MAX = 0.0, 3.0  # sweep range
K_STEPS = 31             # number of K samples
DT = 0.01
STEPS = 8000
TAIL_FRAC = 0.5
SEED = 42

# Frequency distribution
rng_np = np.random.default_rng(SEED)
if DIST == "gaussian":
    omega_np = rng_np.normal(0.0, SIGMA, size=N).astype(np.float64)
elif DIST == "lorentz":
    # Cauchy with gamma=SIGMA (scale), location=0
    omega_np = rng_np.standard_cauchy(size=N).astype(np.float64) * SIGMA
else:
    raise ValueError("Unknown DIST")

omega_xp = xp.asarray(omega_np)

# K sweep
Ks = np.linspace(K_MIN, K_MAX, K_STEPS)
Rs = []
t0 = time.time()
for idx, K in enumerate(Ks):
    R_tail = kuramoto_R_tail(omega_xp, float(K), dt=DT, steps=STEPS, tail_frac=TAIL_FRAC, seed=SEED + idx)
    Rs.append(R_tail)

elapsed = time.time() - t0
Rs = np.array(Rs, dtype=np.float64)

# ------------------------------------------------------------
# 4) Knee/critical-region estimate (simple change-point on discrete derivative)
# ------------------------------------------------------------
dR = np.diff(Rs)
# Use a robust z-score to find the first significant slope jump
mu, sd = float(np.median(dR)), float(np.median(np.abs(dR - np.median(dR))) + 1e-9)
z = (dR - mu) / (sd if sd > 0 else 1.0)
# Index where z first exceeds threshold
Z_THR = 3.0
idx_knee = int(np.argmax(z > Z_THR)) if np.any(z > Z_THR) else int(np.argmax(dR))
K_knee = float(Ks[idx_knee + 1]) if idx_knee + 1 < len(Ks) else float(Ks[-1])

# Optional: estimate "Kc_susc" via max slope
idx_susc = int(np.argmax(dR))
Kc_susc = float(Ks[idx_susc + 1]) if idx_susc + 1 < len(Ks) else float(Ks[-1])

# ------------------------------------------------------------
# 5) Save artifacts
# ------------------------------------------------------------
# CSV
import csv
csv_path = RUN_DIR / "kuramoto_sweep.csv"
with open(csv_path, "w", newline="") as f:
    w = csv.writer(f)
    w.writerow(["K", "R_tail"])
    for k, r in zip(Ks, Rs):
        w.writerow([f"{k:.6f}", f"{r:.6f}"])

# Plot (single plot, matplotlib only, no styles/colors set)
fig = plt.figure(figsize=(7.2, 4.5), dpi=140)
plt.plot(Ks, Rs, marker="o")
plt.axvline(K_knee, linestyle="--", label=f"K_knee≈{K_knee:.3f}")
plt.axvline(Kc_susc, linestyle=":", label=f"Kc_susc≈{Kc_susc:.3f}")
plt.xlabel("K (coupling)")
plt.ylabel("R (order parameter, tail mean)")
plt.title(f"Kuramoto Sweep (N={N}, {DIST}, σ={SIGMA})")
plt.legend()
plt.tight_layout()
png_path = RUN_DIR / "kuramoto_sweep.png"
plt.savefig(png_path)
plt.close(fig)

# Meta JSON
meta = {
    "run_id": RUN_ID,
    "root": str(ROOT),
    "artifacts_dir": str(RUN_DIR),
    "env": {
        "python": sys.version.split()[0],
        "platform": platform.platform(),
        "gpu_backend": "cupy" if HAS_CP else "numpy",
    },
    "params": {
        "N": N, "sigma": SIGMA, "dist": DIST,
        "K_min": K_MIN, "K_max": K_MAX, "K_steps": K_STEPS,
        "dt": DT, "steps": STEPS, "tail_frac": TAIL_FRAC, "seed": SEED
    },
    "results": {
        "K_knee": K_knee,
        "Kc_susc": Kc_susc,
        "elapsed_sec": elapsed
    },
    "files": {
        "csv": str(csv_path),
        "png": str(png_path)
    }
}
with open(RUN_DIR / "run_meta.json", "w") as f:
    json.dump(meta, f, indent=2)

print("=== CNT Quickstart — Completed ===")
print(f"Backend     : {'CuPy (GPU)' if HAS_CP else 'NumPy (CPU)'}")
print(f"Artifacts   : {RUN_DIR}")
print(f"K_knee      : {K_knee:.4f}")
print(f"Kc_susc     : {Kc_susc:.4f}")
print(f"Elapsed (s) : {elapsed:.2f}")


C:\Users\caleb\AppData\Local\Temp\ipykernel_7828\653980359.py:20: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  RUN_ID = dt.datetime.utcnow().strftime("%Y%m%d-%H%M%SZ") + "-" + uuid.uuid4().hex[:6]


=== CNT Quickstart — Completed ===
Backend     : CuPy (GPU)
Artifacts   : E:\CNT\artifacts\cnt_quickstart\20251110-031005Z-15c224
K_knee      : 1.5000
Kc_susc     : 1.5000
Elapsed (s) : 185.44


In [2]:
# === CNT Quickstart v2 — Faster sweep, UTC-safe RUN_ID, FP32 option ===
import os, json, math, time, sys, platform, uuid, datetime as dt
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# ----------------------------
# 0) Paths & run id (UTC-safe)
# ----------------------------
CANDIDATES = [Path("E:/CNT"), Path("E:\\CNT"), Path("./CNT")]
ROOT = next((p for p in CANDIDATES if p.exists()), Path(".").resolve())
ART = (ROOT / "artifacts").resolve()
RUN_ID = dt.datetime.now(dt.timezone.utc).strftime("%Y%m%d-%H%M%SZ") + "-" + uuid.uuid4().hex[:6]
RUN_DIR = (ART / f"cnt_quickstart/{RUN_ID}")
RUN_DIR.mkdir(parents=True, exist_ok=True)

# ----------------------------
# 1) Backend & dtype
# ----------------------------
try:
    import cupy as cp  # type: ignore
    _ = cp.zeros((1,))
    xp = cp
    HAS_CP = True
except Exception:
    xp = np
    HAS_CP = False

FP32 = True  # set False for FP64 fidelity; True is faster esp. on GPU
DTYPE = np.float32 if FP32 else np.float64

def to_np(a):
    return cp.asnumpy(a) if HAS_CP else a

# ----------------------------
# 2) Kuramoto (batched stepping)
# ----------------------------
def kuramoto_R_tail_batched(omega_xp, K, *, dt=0.01, steps=6000, block=5, tail_frac=0.5, seed=0):
    """
    Integrate dθ/dt = ω + K * Im(z_mean * e^{-iθ}) with block sub-steps per loop
    to reduce Python overhead. Returns tail-averaged R.
    """
    rs = xp.random.RandomState(seed) if not HAS_CP else xp.random.RandomState(seed)
    N = omega_xp.shape[0]
    theta = (2 * xp.pi) * rs.rand(N, dtype=DTYPE)
    steps = int(steps)
    block = max(1, int(block))
    outer = (steps + block - 1) // block
    tail_start = int((1.0 - tail_frac) * steps)

    R_sum, R_cnt = 0.0, 0
    t = 0
    for _ in range(outer):
        # do up to 'block' fine steps within this outer iteration
        b_left = min(block, steps - t)
        for _ in range(b_left):
            z = xp.exp(1j * theta)
            z_mean = z.mean()
            # instantaneous R
            if t >= tail_start:
                R_sum += float(xp.abs(z_mean))
                R_cnt += 1
            coupling = xp.imag(z_mean * xp.exp(-1j * theta))
            theta = theta + dt * (omega_xp + K * coupling)
            # periodic rewrap to avoid drift
            if (t & 255) == 0:
                theta = xp.mod(theta, 2 * xp.pi)
            t += 1
        # cheap barrier on GPU to avoid async backlog
        if HAS_CP:
            cp.cuda.Stream.null.synchronize()
    return R_sum / max(R_cnt, 1)

# ----------------------------
# 3) Experiment spec
# ----------------------------
N = 1024                  # oscillators
SIGMA = 1.0               # freq scale (std for Gaussian, gamma for Lorentz)
DIST = "gaussian"         # 'gaussian' or 'lorentz'
K_MIN, K_MAX = 0.0, 3.0
K_STEPS = 31
DTINTEGRATE = 0.01
STEPS = 6000              # you used 8000; 6000 with BLOCK=5 usually suffices
BLOCK = 5                 # sub-steps per Python loop (higher = fewer Python iters)
TAIL_FRAC = 0.5
SEED = 42

# ----------------------------
# 4) Frequencies & arrays
# ----------------------------
rng_np = np.random.default_rng(SEED)
if DIST == "gaussian":
    omega_np = rng_np.normal(0.0, SIGMA, size=N).astype(DTYPE)
elif DIST == "lorentz":
    omega_np = (rng_np.standard_cauchy(size=N).astype(DTYPE)) * SIGMA
else:
    raise ValueError("Unknown DIST")

# move to backend with desired dtype
omega_xp = xp.asarray(omega_np, dtype=DTYPE)

Ks = np.linspace(K_MIN, K_MAX, K_STEPS, dtype=np.float64)  # keep K in FP64 for sweep grid
Rs = np.empty_like(Ks, dtype=np.float64)

# ----------------------------
# 5) Sweep
# ----------------------------
t0 = time.time()
for i, K in enumerate(Ks):
    Rs[i] = kuramoto_R_tail_batched(
        omega_xp, float(K),
        dt=DTINTEGRATE, steps=STEPS, block=BLOCK,
        tail_frac=TAIL_FRAC, seed=SEED + i
    )
elapsed = time.time() - t0

# ----------------------------
# 6) Knee estimates
# ----------------------------
dR = np.diff(Rs)
mu = float(np.median(dR))
sd = float(np.median(np.abs(dR - mu)) + 1e-9)
z = (dR - mu) / (sd if sd > 0 else 1.0)
Z_THR = 3.0
idx_knee = int(np.argmax(z > Z_THR)) if np.any(z > Z_THR) else int(np.argmax(dR))
K_knee = float(Ks[min(idx_knee + 1, len(Ks) - 1)])
idx_susc = int(np.argmax(dR))
Kc_susc = float(Ks[min(idx_susc + 1, len(Ks) - 1)])

# ----------------------------
# 7) Artifacts
# ----------------------------
# CSV
csv_path = RUN_DIR / "kuramoto_sweep.csv"
with open(csv_path, "w", newline="") as f:
    f.write("K,R_tail\n")
    for k, r in zip(Ks, Rs):
        f.write(f"{k:.6f},{r:.6f}\n")

# Plot 1: R(K)
fig1 = plt.figure(figsize=(7.2, 4.5), dpi=140)
plt.plot(Ks, Rs, marker="o")
plt.axvline(K_knee, linestyle="--", label=f"K_knee≈{K_knee:.3f}")
plt.axvline(Kc_susc, linestyle=":", label=f"Kc_susc≈{Kc_susc:.3f}")
plt.xlabel("K (coupling)")
plt.ylabel("R (order parameter, tail mean)")
plt.title(f"Kuramoto Sweep (N={N}, {DIST}, σ/γ={SIGMA}, dtype={'fp32' if FP32 else 'fp64'})")
plt.legend()
plt.tight_layout()
png1 = RUN_DIR / "kuramoto_sweep.png"
plt.savefig(png1)
plt.close(fig1)

# Plot 2: slope dR/dK
fig2 = plt.figure(figsize=(7.2, 3.8), dpi=140)
dK = np.diff(Ks)
plt.plot(Ks[:-1] + 0.5 * dK, dR, marker=".")
plt.axvline(Kc_susc, linestyle=":", label=f"max slope≈{Kc_susc:.3f}")
plt.xlabel("K")
plt.ylabel("ΔR / ΔK")
plt.title("Discrete slope of R(K)")
plt.legend()
plt.tight_layout()
png2 = RUN_DIR / "kuramoto_slope.png"
plt.savefig(png2)
plt.close(fig2)

# Meta
meta = {
    "run_id": RUN_ID,
    "root": str(ROOT),
    "artifacts_dir": str(RUN_DIR),
    "env": {
        "python": sys.version.split()[0],
        "platform": platform.platform(),
        "backend": "cupy" if HAS_CP else "numpy",
        "dtype": "fp32" if FP32 else "fp64",
    },
    "params": {
        "N": N, "sigma_or_gamma": SIGMA, "dist": DIST,
        "K_min": float(K_MIN), "K_max": float(K_MAX), "K_steps": int(K_STEPS),
        "dt": DTINTEGRATE, "steps": int(STEPS), "block": int(BLOCK),
        "tail_frac": TAIL_FRAC, "seed": SEED
    },
    "results": {
        "K_knee": K_knee,
        "Kc_susc": Kc_susc,
        "elapsed_sec": elapsed
    },
    "files": {
        "csv": str(csv_path),
        "png_R": str(png1),
        "png_slope": str(png2)
    }
}
with open(RUN_DIR / "run_meta.json", "w") as f:
    json.dump(meta, f, indent=2)

print("=== CNT Quickstart v2 — Completed ===")
print(f"Backend     : {'CuPy (GPU)' if HAS_CP else 'NumPy (CPU)'}  | dtype: {'fp32' if FP32 else 'fp64'}")
print(f"Artifacts   : {RUN_DIR}")
print(f"K_knee      : {K_knee:.4f}")
print(f"Kc_susc     : {Kc_susc:.4f}")
print(f"Elapsed (s) : {elapsed:.2f}")


=== CNT Quickstart v2 — Completed ===
Backend     : CuPy (GPU)  | dtype: fp32
Artifacts   : E:\CNT\artifacts\cnt_quickstart\20251110-031541Z-803a33
K_knee      : 1.5000
Kc_susc     : 1.6000
Elapsed (s) : 46.63


In [3]:
# === CNT Finite-Size Scaling — One Cell (GPU/CPU, UTC-safe, fp32/fp64) ===
# Runs Kuramoto sweeps for N in {512,1024,2048}; estimates Kc (max slope),
# fits β from R ~ (K - Kc)^β, and emits clean artifacts.
# Artifacts:
#   E:/CNT/artifacts/cnt_fss/<RUN_ID>/
#     - fss_R_vs_K.png         (R(K) for all N + Kc markers)
#     - fss_slope.png          (dR/dK for all N + max-slope)
#     - fss_beta_fit.png       (log-log fit for β with windowing)
#     - fss_results.csv        (K, R, N rows)
#     - run_meta.json          (params, Kc per N, β estimate, timing)

import os, json, math, time, sys, platform, uuid, datetime as dt
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# ----------------------------
# 0) Paths & run id (UTC-safe)
# ----------------------------
CANDIDATES = [Path("E:/CNT"), Path("E:\\CNT"), Path("./CNT")]
ROOT = next((p for p in CANDIDATES if p.exists()), Path(".").resolve())
ART = (ROOT / "artifacts").resolve()
RUN_ID = dt.datetime.now(dt.timezone.utc).strftime("%Y%m%d-%H%M%SZ") + "-" + uuid.uuid4().hex[:6]
RUN_DIR = (ART / f"cnt_fss/{RUN_ID}")
RUN_DIR.mkdir(parents=True, exist_ok=True)

# ----------------------------
# 1) Backend & dtype
# ----------------------------
try:
    import cupy as cp  # type: ignore
    _ = cp.zeros((1,))
    xp = cp
    HAS_CP = True
except Exception:
    xp = np
    HAS_CP = False

FP32 = True  # True for speed (GPU esp.); False for fp64 fidelity
DTYPE = np.float32 if FP32 else np.float64

def to_np(a):
    return cp.asnumpy(a) if HAS_CP else a

# ----------------------------
# 2) Kuramoto (batched stepping)
# ----------------------------
def kuramoto_R_tail_batched(omega_xp, K, *, dt=0.01, steps=6000, block=5, tail_frac=0.5, seed=0):
    """Integrate dθ/dt = ω + K * Im(z_mean * e^{-iθ}) with 'block' sub-steps per loop.
       Returns tail-averaged R."""
    rs = xp.random.RandomState(seed) if not HAS_CP else xp.random.RandomState(seed)
    N = omega_xp.shape[0]
    theta = (2 * xp.pi) * rs.rand(N, dtype=DTYPE)
    steps = int(steps)
    block = max(1, int(block))
    outer = (steps + block - 1) // block
    tail_start = int((1.0 - tail_frac) * steps)

    R_sum, R_cnt = 0.0, 0
    t = 0
    for _ in range(outer):
        b_left = min(block, steps - t)
        for _ in range(b_left):
            z = xp.exp(1j * theta)
            z_mean = z.mean()
            if t >= tail_start:
                R_sum += float(xp.abs(z_mean))
                R_cnt += 1
            coupling = xp.imag(z_mean * xp.exp(-1j * theta))
            theta = theta + dt * (omega_xp + K * coupling)
            if (t & 255) == 0:
                theta = xp.mod(theta, 2 * xp.pi)
            t += 1
        if HAS_CP:
            cp.cuda.Stream.null.synchronize()
    return R_sum / max(R_cnt, 1)

# ----------------------------
# 3) Experiment spec
# ----------------------------
Ns = [512, 1024, 2048]    # finite-size sweep
SIGMA = 1.0               # std (Gaussian) or gamma (Lorentz)
DIST = "gaussian"         # 'gaussian' or 'lorentz'
K_MIN, K_MAX = 0.0, 3.0
K_STEPS = 49              # finer grid gives crisper slopes
DTINTEGRATE = 0.01
STEPS = 6000
BLOCK = 5
TAIL_FRAC = 0.5
SEED = 42

# Theoretical Kc (reference, not enforced)
if DIST == "gaussian":
    # g(0) = 1/(sigma*sqrt(2π)) → Kc = 2/(π g(0)) = 2 sigma sqrt(2π) / π
    Kc_theory = 2.0 * SIGMA * math.sqrt(2.0 * math.pi) / math.pi
elif DIST == "lorentz":
    # Kc = 2γ
    Kc_theory = 2.0 * SIGMA
else:
    raise ValueError("Unknown DIST")

# ----------------------------
# 4) Sweep for each N
# ----------------------------
rng_np = np.random.default_rng(SEED)
Ks = np.linspace(K_MIN, K_MAX, K_STEPS, dtype=np.float64)
rows = []
Kc_by_N = {}
elapsed_total = 0.0

t0_all = time.time()
for Ni in Ns:
    # frequencies per N (same seed path; larger N draws more)
    if DIST == "gaussian":
        omega_np = rng_np.normal(0.0, SIGMA, size=Ni).astype(DTYPE)
    else:
        omega_np = (rng_np.standard_cauchy(size=Ni).astype(DTYPE)) * SIGMA
    omega_xp = xp.asarray(omega_np, dtype=DTYPE)

    Rs = np.empty_like(Ks, dtype=np.float64)
    t0 = time.time()
    for i, K in enumerate(Ks):
        Rs[i] = kuramoto_R_tail_batched(
            omega_xp, float(K),
            dt=DTINTEGRATE, steps=STEPS, block=BLOCK,
            tail_frac=TAIL_FRAC, seed=SEED + i
        )
    elapsed = time.time() - t0
    elapsed_total += elapsed

    # susceptibility proxy (max slope)
    dR = np.diff(Rs)
    idx_susc = int(np.argmax(dR))
    Kc_susc = float(Ks[min(idx_susc + 1, len(Ks) - 1)])
    Kc_by_N[int(Ni)] = {"Kc_susc": Kc_susc}

    for k, r in zip(Ks, Rs):
        rows.append((int(Ni), float(k), float(r)))

# ----------------------------
# 5) Fit β using near-critical window
# ----------------------------
# Merge all points; for each N, use its Kc_susc
rows_np = np.array(rows, dtype=[("N", np.int32), ("K", np.float64), ("R", np.float64)])

def fit_beta(rows_np, Kc_map, r_min=0.05, r_max=0.6):
    X, Y = [], []
    for (Ni, K, R) in rows_np:
        KcN = Kc_map[int(Ni)]["Kc_susc"]
        if K > KcN and r_min <= R <= r_max:
            x = K - KcN
            if x > 0:
                X.append(math.log(x))
                Y.append(math.log(R))
    if len(X) < 4:  # not enough points; relax window if needed
        # fall back to broader window
        X, Y = [], []
        for (Ni, K, R) in rows_np:
            KcN = Kc_map[int(Ni)]["Kc_susc"]
            if K > KcN and 0.02 <= R <= 0.8:
                x = K - KcN
                if x > 0:
                    X.append(math.log(x))
                    Y.append(math.log(R))
    X = np.array(X, dtype=np.float64)
    Y = np.array(Y, dtype=np.float64)
    if len(X) >= 2:
        m, b = np.polyfit(X, Y, 1)
        beta = m  # slope in log-log; R ~ (K-Kc)^β
        return beta, (m, b), X, Y
    return float("nan"), (float("nan"), float("nan")), np.array([]), np.array([])

beta_est, (m_fit, b_fit), X_ll, Y_ll = fit_beta(rows_np, Kc_by_N)

# ----------------------------
# 6) Save CSV
# ----------------------------
csv_path = RUN_DIR / "fss_results.csv"
with open(csv_path, "w", newline="") as f:
    f.write("N,K,R\n")
    for Nval, Kval, Rval in rows:
        f.write(f"{Nval},{Kval:.6f},{Rval:.6f}\n")

# ----------------------------
# 7) Plots (matplotlib, single plots, no custom colors/styles)
# ----------------------------
# R vs K
fig1 = plt.figure(figsize=(7.8, 4.6), dpi=140)
for Ni in Ns:
    mask = rows_np["N"] == Ni
    Ki = rows_np["K"][mask]
    Ri = rows_np["R"][mask]
    plt.plot(Ki, Ri, marker="o", label=f"N={Ni}")
    plt.axvline(Kc_by_N[int(Ni)]["Kc_susc"], linestyle=":", label=f"Kc(N={Ni})≈{Kc_by_N[int(Ni)]['Kc_susc']:.3f}")
plt.axvline(Kc_theory, linestyle="--", label=f"Kc_theory≈{Kc_theory:.3f}")
plt.xlabel("K (coupling)")
plt.ylabel("R (order parameter, tail mean)")
plt.title(f"Kuramoto Finite-Size Sweep ({DIST}, σ/γ={SIGMA}, dtype={'fp32' if FP32 else 'fp64'})")
plt.legend()
plt.tight_layout()
png1 = RUN_DIR / "fss_R_vs_K.png"
plt.savefig(png1); plt.close(fig1)

# dR/dK
fig2 = plt.figure(figsize=(7.8, 4.2), dpi=140)
for Ni in Ns:
    mask = rows_np["N"] == Ni
    Ki = rows_np["K"][mask]
    Ri = rows_np["R"][mask]
    dR = np.diff(Ri)
    dK = np.diff(Ki)
    Kmid = Ki[:-1] + 0.5 * dK
    plt.plot(Kmid, dR / dK, marker=".", label=f"N={Ni}")
    plt.axvline(Kc_by_N[int(Ni)]["Kc_susc"], linestyle=":", label=f"Kc(N={Ni})")
plt.xlabel("K")
plt.ylabel("dR/dK (discrete)")
plt.title("Discrete slope of R(K)")
plt.legend()
plt.tight_layout()
png2 = RUN_DIR / "fss_slope.png"
plt.savefig(png2); plt.close(fig2)

# β fit (log-log)
fig3 = plt.figure(figsize=(6.4, 4.6), dpi=140)
if len(X_ll) >= 2:
    x_grid = np.linspace(X_ll.min(), X_ll.max(), 200)
    y_grid = m_fit * x_grid + b_fit
    plt.plot(X_ll, Y_ll, marker="o", linestyle="", label="data (windowed)")
    plt.plot(x_grid, y_grid, label=f"fit: β≈{m_fit:.3f}")
    plt.xlabel("log(K - Kc(N))")
    plt.ylabel("log R")
    plt.title("Critical scaling fit")
    plt.legend()
else:
    plt.text(0.5, 0.5, "Insufficient points for β fit", ha="center", va="center")
    plt.axis("off")
plt.tight_layout()
png3 = RUN_DIR / "fss_beta_fit.png"
plt.savefig(png3); plt.close(fig3)

elapsed_all = time.time() - t0_all

# ----------------------------
# 8) Meta JSON
# ----------------------------
meta = {
    "run_id": RUN_ID,
    "artifacts_dir": str(RUN_DIR),
    "env": {
        "python": sys.version.split()[0],
        "platform": platform.platform(),
        "backend": "cupy" if HAS_CP else "numpy",
        "dtype": "fp32" if FP32 else "fp64",
    },
    "dist": DIST,
    "sigma_or_gamma": SIGMA,
    "theory": {"Kc_theory": Kc_theory},
    "grid": {"K_min": float(K_MIN), "K_max": float(K_MAX), "K_steps": int(K_STEPS)},
    "integrator": {"dt": DTINTEGRATE, "steps": int(STEPS), "block": int(BLOCK), "tail_frac": float(TAIL_FRAC)},
    "sizes": Ns,
    "results": {
        "Kc_by_N": Kc_by_N,
        "beta_est": float(beta_est),
        "elapsed_sec_total": elapsed_all
    },
    "files": {
        "csv": str(csv_path),
        "png_R_vs_K": str(png1),
        "png_slope": str(png2),
        "png_beta_fit": str(png3)
    }
}
with open(RUN_DIR / "run_meta.json", "w") as f:
    json.dump(meta, f, indent=2)

print("=== CNT Finite-Size Scaling — Completed ===")
print(f"Backend     : {'CuPy (GPU)' if HAS_CP else 'NumPy (CPU)'} | dtype: {'fp32' if FP32 else 'fp64'}")
print(f"Artifacts   : {RUN_DIR}")
for Ni in Ns:
    print(f"N={Ni:4d} → Kc_susc≈{Kc_by_N[int(Ni)]['Kc_susc']:.4f}")
print(f"Theory Kc   : {Kc_theory:.4f} ({DIST}, σ/γ={SIGMA})")
print(f"β estimate  : {beta_est:.3f}")
print(f"Elapsed (s) : {elapsed_all:.2f}")


=== CNT Finite-Size Scaling — Completed ===
Backend     : CuPy (GPU) | dtype: fp32
Artifacts   : E:\CNT\artifacts\cnt_fss\20251110-033047Z-b80f75
N= 512 → Kc_susc≈1.5000
N=1024 → Kc_susc≈1.6250
N=2048 → Kc_susc≈1.6875
Theory Kc   : 1.5958 (gaussian, σ/γ=1.0)
β estimate  : 0.169
Elapsed (s) : 200.72


In [4]:
# === CNT Critical Refinement — Multi-seed + Binder crossing + β fit (one cell) ===
# Artifacts -> E:/CNT/artifacts/cnt_crit_refine/<RUN_ID>/

import os, json, math, time, sys, platform, uuid, datetime as dt
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# ----------------------------
# 0) Paths & UTC-safe run id
# ----------------------------
CANDIDATES = [Path("E:/CNT"), Path("E:\\CNT"), Path("./CNT")]
ROOT = next((p for p in CANDIDATES if p.exists()), Path(".").resolve())
ART = (ROOT / "artifacts").resolve()
RUN_ID = dt.datetime.now(dt.timezone.utc).strftime("%Y%m%d-%H%M%SZ") + "-" + uuid.uuid4().hex[:6]
RUN_DIR = (ART / f"cnt_crit_refine/{RUN_ID}")
RUN_DIR.mkdir(parents=True, exist_ok=True)

# ----------------------------
# 1) Backend & dtype
# ----------------------------
try:
    import cupy as cp  # type: ignore
    _ = cp.zeros((1,))
    xp = cp
    HAS_CP = True
except Exception:
    xp = np
    HAS_CP = False

FP32 = True    # speed; set False for fp64 fidelity
DTYPE = np.float32 if FP32 else np.float64

def to_np(a): return cp.asnumpy(a) if HAS_CP else a

# ----------------------------
# 2) Kuramoto integrator (returns tail moments)
# ----------------------------
def kuramoto_tail_moments(omega_xp, K, *, dt=0.01, steps=8000, block=5, tail_frac=0.5, seed=0):
    """
    Integrate dθ/dt = ω + K * Im(z_mean e^{-iθ}); return tail-averaged
    moments of R: <R>, <R^2>, <R^4> taken over time within the tail window.
    """
    rs = xp.random.RandomState(seed) if not HAS_CP else xp.random.RandomState(seed)
    N = omega_xp.shape[0]
    theta = (2 * xp.pi) * rs.rand(N, dtype=DTYPE)
    steps = int(steps); block = max(1, int(block))
    outer = (steps + block - 1) // block
    t, tail_start = 0, int((1.0 - tail_frac) * steps)

    s1 = 0.0; s2 = 0.0; s4 = 0.0; cnt = 0
    for _ in range(outer):
        b = min(block, steps - t)
        for _ in range(b):
            z = xp.exp(1j * theta); z_mean = z.mean()
            if t >= tail_start:
                Rt = float(xp.abs(z_mean))
                s1 += Rt; s2 += Rt*Rt; s4 += (Rt*Rt)*(Rt*Rt); cnt += 1
            coupling = xp.imag(z_mean * xp.exp(-1j * theta))
            theta = theta + dt * (omega_xp + K * coupling)
            if (t & 255) == 0:
                theta = xp.mod(theta, 2 * xp.pi)
            t += 1
        if HAS_CP: cp.cuda.Stream.null.synchronize()
    if cnt == 0: return 0.0, 0.0, 0.0
    m1 = s1 / cnt
    m2 = s2 / cnt
    m4 = s4 / cnt
    return m1, m2, m4

# ----------------------------
# 3) Experiment spec
# ----------------------------
Ns = [512, 1024, 2048]
SEEDS = [101, 202, 303, 404, 505]   # multi-seed averaging
DIST = "gaussian"                    # 'gaussian' or 'lorentz'
SIGMA = 1.0
DT = 0.01
STEPS = 8000
BLOCK = 5
TAIL_FRAC = 0.5

# theory reference
if DIST == "gaussian":
    Kc_theory = 2.0 * SIGMA * math.sqrt(2.0 * math.pi) / math.pi
elif DIST == "lorentz":
    Kc_theory = 2.0 * SIGMA
else:
    raise ValueError("Unknown DIST")

# coarse grid then adaptive zoom
K_MIN, K_MAX = 0.8, 2.2
K_STEPS_COARSE = 41
COARSE_K = np.linspace(K_MIN, K_MAX, K_STEPS_COARSE)

# ----------------------------
# 4) Sweep (coarse) → provisional Kc_susc(N)
# ----------------------------
rng_np = np.random.default_rng(42)
data = {}   # N -> dict with arrays and results

t_all = time.time()
for Ni in Ns:
    # draw frequencies
    if DIST == "gaussian":
        omega_np = rng_np.normal(0.0, SIGMA, size=Ni).astype(DTYPE)
    else:
        omega_np = (rng_np.standard_cauchy(size=Ni).astype(DTYPE)) * SIGMA
    omega_xp = xp.asarray(omega_np, dtype=DTYPE)

    R_mean = np.zeros_like(COARSE_K, dtype=np.float64)
    R2_mean = np.zeros_like(COARSE_K, dtype=np.float64)
    R4_mean = np.zeros_like(COARSE_K, dtype=np.float64)

    for si, seed in enumerate(SEEDS):
        for i, K in enumerate(COARSE_K):
            m1, m2, m4 = kuramoto_tail_moments(
                omega_xp, float(K), dt=DT, steps=STEPS, block=BLOCK, tail_frac=TAIL_FRAC, seed=seed+i
            )
            # online average
            R_mean[i] = (R_mean[i]*si + m1) / (si+1)
            R2_mean[i] = (R2_mean[i]*si + m2) / (si+1)
            R4_mean[i] = (R4_mean[i]*si + m4) / (si+1)

    # susceptibility proxy (max discrete slope of <R>)
    dR = np.diff(R_mean); dK = np.diff(COARSE_K)
    slope = dR / dK
    idx_susc = int(np.argmax(slope))
    Kc_susc = float(COARSE_K[min(idx_susc + 1, len(COARSE_K)-1)])

    data[Ni] = {
        "K": COARSE_K.copy(), "R": R_mean, "R2": R2_mean, "R4": R4_mean,
        "Kc_susc_coarse": Kc_susc
    }

# ----------------------------
# 5) Adaptive zoom around each Kc_susc(N)
# ----------------------------
def refine_grid(Kc_guess, span=0.3, points=61):
    lo = max(K_MIN, Kc_guess - span)
    hi = min(K_MAX, Kc_guess + span)
    return np.linspace(lo, hi, points)

for Ni in Ns:
    Kc_guess = data[Ni]["Kc_susc_coarse"]
    K_ref = refine_grid(Kc_guess, span=0.25, points=81)
    Rm = np.zeros_like(K_ref, dtype=np.float64)
    R2 = np.zeros_like(K_ref, dtype=np.float64)
    R4 = np.zeros_like(K_ref, dtype=np.float64)

    # reuse frequencies to keep disorder fixed across seeds
    if DIST == "gaussian":
        omega_np = rng_np.normal(0.0, SIGMA, size=Ni).astype(DTYPE)
    else:
        omega_np = (rng_np.standard_cauchy(size=Ni).astype(DTYPE)) * SIGMA
    omega_xp = xp.asarray(omega_np, dtype=DTYPE)

    for si, seed in enumerate(SEEDS):
        for i, K in enumerate(K_ref):
            m1, m2, m4 = kuramoto_tail_moments(
                omega_xp, float(K), dt=DT, steps=STEPS, block=BLOCK, tail_frac=TAIL_FRAC, seed=seed+i
            )
            Rm[i]  = (Rm[i]*si  + m1) / (si+1)
            R2[i]  = (R2[i]*si  + m2) / (si+1)
            R4[i]  = (R4[i]*si  + m4) / (si+1)

    dR = np.diff(Rm); dK = np.diff(K_ref)
    slope = dR / dK
    idx_susc = int(np.argmax(slope))
    Kc_fine = float(K_ref[min(idx_susc + 1, len(K_ref)-1)])

    data[Ni].update({"K": K_ref, "R": Rm, "R2": R2, "R4": R4, "Kc_susc_fine": Kc_fine})

# ----------------------------
# 6) Binder-like cumulant and crossings
# ----------------------------
def binder_U(R2, R4):
    # U = 1 - <R^4> / (3 <R^2>^2)
    eps = 1e-12
    return 1.0 - (R4 / (3.0 * np.maximum(R2*R2, eps)))

def interp_crossing(K1, U1, K2, U2):
    # find K where U1(K) - U2(K) = 0 via linear interpolation between nearest sign change
    D = U1 - U2
    s = np.sign(D)
    idx = np.where(np.diff(s) != 0)[0]
    if len(idx) == 0:
        # fallback: nearest approach
        j = int(np.argmin(np.abs(D)))
        return float(K1[j])
    j = int(idx[np.argmin(np.abs(D[idx]))])
    x0, x1 = K1[j], K1[j+1]
    y0, y1 = D[j], D[j+1]
    if (y1 - y0) == 0: return float(x0)
    return float(x0 - y0 * (x1 - x0) / (y1 - y0))

U_by_N = {}
for Ni in Ns:
    U_by_N[Ni] = binder_U(data[Ni]["R2"], data[Ni]["R4"])

crossings = []
pairs = []
for i in range(len(Ns)-1):
    N1, N2 = Ns[i], Ns[i+1]
    # align on common K grid by interpolation to N2's grid
    K2 = data[N2]["K"]
    U1i = np.interp(K2, data[N1]["K"], U_by_N[N1])
    U2i = U_by_N[N2]
    Kc_pair = interp_crossing(K2, U1i, K2, U2i)
    crossings.append((N1, N2, Kc_pair))
    pairs.append((N1, N2))

# Estimate Kc(∞) via linear fit vs 1/N_mid, where N_mid is harmonic mean of pair
N_mid = np.array([2/(1/N1 + 1/N2) for (N1, N2, _) in crossings], dtype=np.float64)
Kc_pair_vals = np.array([k for (_,_,k) in crossings], dtype=np.float64)
X = 1.0 / N_mid
if len(X) >= 2:
    a,b = np.polyfit(X, Kc_pair_vals, 1)  # Kc ~ a*(1/N_mid) + b
    Kc_infty = float(b)
else:
    Kc_infty = float(np.mean(Kc_pair_vals)) if len(Kc_pair_vals)>0 else float('nan')

# ----------------------------
# 7) β fit using shared Kc(∞)
# ----------------------------
def fit_beta_shared_Kc(Kc_star, rows, r_lo=0.06, r_hi=0.6):
    X_ll, Y_ll = [], []
    for (N, K, R) in rows:
        if K > Kc_star and r_lo <= R <= r_hi:
            x = K - Kc_star
            if x > 0:
                X_ll.append(math.log(x)); Y_ll.append(math.log(R))
    X_ll, Y_ll = np.array(X_ll), np.array(Y_ll)
    if len(X_ll) >= 2:
        m,b = np.polyfit(X_ll, Y_ll, 1)
        return float(m), (m,b), X_ll, Y_ll
    return float('nan'), (float('nan'), float('nan')), np.array([]), np.array([])

rows_all = []
for Ni in Ns:
    K = data[Ni]["K"]; R = data[Ni]["R"]
    for k, r in zip(K, R):
        rows_all.append((Ni, float(k), float(r)))

beta_est, (m_fit, b_fit), X_ll, Y_ll = fit_beta_shared_Kc(Kc_infty, rows_all)

elapsed_all = time.time() - t_all

# ----------------------------
# 8) Save CSV + meta + plots
# ----------------------------
# CSV per N
for Ni in Ns:
    with open(RUN_DIR / f"crit_{Ni}.csv", "w") as f:
        f.write("K,R,R2,R4,U\n")
        K = data[Ni]["K"]; R = data[Ni]["R"]; R2 = data[Ni]["R2"]; R4 = data[Ni]["R4"]
        U = binder_U(R2,R4)
        for k,r,r2,r4,u in zip(K,R,R2,R4,U):
            f.write(f"{k:.6f},{r:.6f},{r2:.6f},{r4:.6f},{u:.6f}\n")

meta = {
    "run_id": RUN_ID,
    "artifacts_dir": str(RUN_DIR),
    "env": {
        "python": sys.version.split()[0],
        "platform": platform.platform(),
        "backend": "cupy" if HAS_CP else "numpy",
        "dtype": "fp32" if FP32 else "fp64",
    },
    "model": {"dist": DIST, "sigma_or_gamma": SIGMA},
    "integrator": {"dt": DT, "steps": STEPS, "block": BLOCK, "tail_frac": TAIL_FRAC},
    "sizes": Ns,
    "seeds": SEEDS,
    "coarse_grid": {"K_min": float(K_MIN), "K_max": float(K_MAX), "steps": int(K_STEPS_COARSE)},
    "theory": {"Kc_theory": Kc_theory},
    "results": {
        "Kc_susc_coarse": {int(N): data[N]["Kc_susc_coarse"] for N in Ns},
        "Kc_susc_fine": {int(N): data[N]["Kc_susc_fine"] for N in Ns},
        "binder_crossings": [{"N1": int(N1), "N2": int(N2), "Kc_pair": float(k)} for (N1,N2,k) in crossings],
        "Kc_infty": Kc_infty,
        "beta_est": beta_est,
        "elapsed_sec": elapsed_all
    }
}
with open(RUN_DIR / "run_meta.json", "w") as f:
    json.dump(meta, f, indent=2)

# Plots
# (1) R(K) with Kc markers and Kc_infty
fig1 = plt.figure(figsize=(8.0, 4.8), dpi=140)
for Ni in Ns:
    K = data[Ni]["K"]; R = data[Ni]["R"]
    plt.plot(K, R, marker="o", label=f"N={Ni}")
    plt.axvline(data[Ni]["Kc_susc_fine"], linestyle=":", label=f"Kc_susc(N={Ni})≈{data[Ni]['Kc_susc_fine']:.3f}")
plt.axvline(Kc_infty, linestyle="--", label=f"Kc(∞)≈{Kc_infty:.3f}")
plt.axvline(Kc_theory, linestyle="-.", label=f"Kc_theory≈{Kc_theory:.3f}")
plt.xlabel("K"); plt.ylabel("<R> (tail mean)")
plt.title("Kuramoto near-critical (multi-seed, refined)")
plt.legend(); plt.tight_layout()
plt.savefig(RUN_DIR / "crit_R_vs_K.png"); plt.close(fig1)

# (2) Binder U(K) for all N + pairwise crossing Kc(N1,N2)
fig2 = plt.figure(figsize=(8.0, 4.6), dpi=140)
for Ni in Ns:
    K = data[Ni]["K"]; U = binder_U(data[Ni]["R2"], data[Ni]["R4"])
    plt.plot(K, U, marker=".", label=f"N={Ni}")
for (N1,N2,kc) in crossings:
    plt.axvline(kc, linestyle=":", label=f"Kc({N1},{N2})≈{kc:.3f}")
plt.axvline(Kc_infty, linestyle="--", label=f"Kc(∞)≈{Kc_infty:.3f}")
plt.xlabel("K"); plt.ylabel("Binder-like U")
plt.title("Binder crossings and Kc(∞)")
plt.legend(); plt.tight_layout()
plt.savefig(RUN_DIR / "crit_binder.png"); plt.close(fig2)

# (3) β fit with shared Kc(∞)
fig3 = plt.figure(figsize=(6.4, 4.8), dpi=140)
if len(X_ll) >= 2:
    xg = np.linspace(X_ll.min(), X_ll.max(), 200)
    yg = m_fit * xg + b_fit
    plt.plot(X_ll, Y_ll, marker="o", linestyle="", label="data window")
    plt.plot(xg, yg, label=f"fit β≈{m_fit:.3f}")
    plt.xlabel("log(K - Kc(∞))"); plt.ylabel("log <R>")
    plt.title("Critical scaling with shared Kc(∞)")
    plt.legend()
else:
    plt.text(0.5, 0.5, "Insufficient points for β fit", ha="center", va="center")
    plt.axis("off")
plt.tight_layout()
plt.savefig(RUN_DIR / "crit_beta_fit.png"); plt.close(fig3)

print("=== CNT Critical Refinement — Completed ===")
print(f"Artifacts : {RUN_DIR}")
print(f"Theory Kc : {Kc_theory:.4f}  |  Kc(∞)≈{Kc_infty:.4f}")
for Ni in Ns:
    print(f"N={Ni} : Kc_susc_coarse≈{data[Ni]['Kc_susc_coarse']:.4f}  |  Kc_susc_fine≈{data[Ni]['Kc_susc_fine']:.4f}")
for (N1,N2,kc) in crossings:
    print(f"Crossing(N={N1},{N2}) → Kc≈{kc:.4f}")
print(f"β (shared Kc) ≈ {beta_est:.3f}")
print(f"Elapsed (s): {elapsed_all:.2f}")


=== CNT Critical Refinement — Completed ===
Artifacts : E:\CNT\artifacts\cnt_crit_refine\20251110-033907Z-18bc53
Theory Kc : 1.5958  |  Kc(∞)≈1.8958
N=512 : Kc_susc_coarse≈1.5000  |  Kc_susc_fine≈1.6375
N=1024 : Kc_susc_coarse≈1.5700  |  Kc_susc_fine≈1.6263
N=2048 : Kc_susc_coarse≈1.6750  |  Kc_susc_fine≈1.5063
Crossing(N=512,1024) → Kc≈1.6926
Crossing(N=1024,2048) → Kc≈1.7942
β (shared Kc) ≈ nan
Elapsed (s): 3131.34


In [ ]:
# === CNT Critical Refinement v2 — nested disorder, fp64, parabolic crossings, robust β ===
# Artifacts → E:/CNT/artifacts/cnt_crit_refine_v2/<RUN_ID>/

import os, json, math, time, sys, platform, uuid, datetime as dt
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# ---------- paths ----------
CANDIDATES = [Path("E:/CNT"), Path("E:\\CNT"), Path("./CNT")]
ROOT = next((p for p in CANDIDATES if p.exists()), Path(".").resolve())
ART = (ROOT / "artifacts").resolve()
RUN_ID = dt.datetime.now(dt.timezone.utc).strftime("%Y%m%d-%H%M%SZ") + "-" + uuid.uuid4().hex[:6]
RUN_DIR = (ART / f"cnt_crit_refine_v2/{RUN_ID}")
RUN_DIR.mkdir(parents=True, exist_ok=True)

# ---------- backend ----------
try:
    import cupy as cp
    _ = cp.zeros((1,))
    xp, HAS_CP = cp, True
except Exception:
    xp, HAS_CP = np, False

DTYPE = np.float64   # fp64 for stable moments
def to_np(a): return cp.asnumpy(a) if HAS_CP else a

# ---------- integrator: tail moments ----------
def kuramoto_tail_moments(omega_xp, K, *, dt=0.01, steps=12000, block=10, tail_frac=0.6, seed=0):
    rs = xp.random.RandomState(seed) if not HAS_CP else xp.random.RandomState(seed)
    N = omega_xp.shape[0]
    theta = (2 * xp.pi) * rs.rand(N, dtype=DTYPE)
    steps = int(steps); block = max(1, int(block))
    outer = (steps + block - 1) // block
    t, tail_start = 0, int((1.0 - tail_frac) * steps)
    s1=s2=s4=0.0; cnt=0
    for _ in range(outer):
        b = min(block, steps - t)
        for _ in range(b):
            z_mean = xp.exp(1j*theta).mean()
            if t >= tail_start:
                R = float(abs(z_mean))
                s1 += R; s2 += R*R; s4 += (R*R)*(R*R); cnt += 1
            coupling = xp.imag(z_mean * xp.exp(-1j*theta))
            theta = theta + dt*(omega_xp + K*coupling)
            if (t & 255) == 0:
                theta = xp.mod(theta, 2*xp.pi)
            t += 1
        if HAS_CP: cp.cuda.Stream.null.synchronize()
    if cnt == 0: return 0.0,0.0,0.0
    m1 = s1/cnt; m2 = s2/cnt; m4 = s4/cnt
    return m1, m2, m4

# ---------- spec ----------
SIGMA = 1.0
DIST  = "gaussian"  # or 'lorentz'
Ns_try = [512, 1024, 2048, 4096]     # will auto-drop 4096 if OOM
SEEDS = [101,202,303,404,505,606,707]  # 7 seeds
DT = 0.01; STEPS = 12000; BLOCK = 10; TAIL_FRAC = 0.6

# theory
if DIST=="gaussian":
    Kc_theory = 2.0 * SIGMA * math.sqrt(2.0*math.pi) / math.pi   # ≈1.59577
else:
    Kc_theory = 2.0 * SIGMA

# grids
K_MIN, K_MAX = 1.1, 2.1
K_FINE = np.linspace(K_MIN, K_MAX, 241)  # dense common grid for all Ns

# ---------- master disorder (nested) ----------
N_max = max(Ns_try)
rng = np.random.default_rng(42)
if DIST=="gaussian":
    omega_master = rng.normal(0.0, SIGMA, size=N_max).astype(DTYPE)
else:
    omega_master = (rng.standard_cauchy(size=N_max).astype(DTYPE))*SIGMA

# safely downselect Ns based on memory
Ns = []
for N in Ns_try:
    try:
        w = xp.asarray(omega_master[:N], dtype=DTYPE)
        _ = w.sum()  # touch
        Ns.append(N)
    except Exception:
        pass
assert len(Ns)>0, "No N fits GPU/CPU memory."

# ---------- runs ----------
def binder_U(R2, R4):
    eps = 1e-12
    return 1.0 - (R4 / (3.0*np.maximum(R2*R2, eps)))

def multi_seed_curve(omega_xp, Ks, seeds):
    Rm = np.zeros_like(Ks, dtype=np.float64)
    R2 = np.zeros_like(Ks, dtype=np.float64)
    R4 = np.zeros_like(Ks, dtype=np.float64)
    for si, sd in enumerate(seeds):
        for i, K in enumerate(Ks):
            m1,m2,m4 = kuramoto_tail_moments(omega_xp, float(K),
                                             dt=DT, steps=STEPS, block=BLOCK, tail_frac=TAIL_FRAC, seed=sd+i)
            Rm[i] = (Rm[i]*si + m1)/(si+1)
            R2[i] = (R2[i]*si + m2)/(si+1)
            R4[i] = (R4[i]*si + m4)/(si+1)
    return Rm, R2, R4

data = {}
t0 = time.time()
for N in Ns:
    omega_xp = xp.asarray(omega_master[:N], dtype=DTYPE)
    Rm,R2,R4 = multi_seed_curve(omega_xp, K_FINE, SEEDS)
    dR = np.diff(Rm)/np.diff(K_FINE)
    k_susc = float(K_FINE[np.argmax(dR)+1])
    data[N] = {"K":K_FINE.copy(), "R":Rm, "R2":R2, "R4":R4, "Kc_susc":k_susc}
elapsed_sim = time.time()-t0

# ---------- parabolic crossing refinement ----------
def parabolic_crossing(K, U1, U2, window=5):
    D = U1 - U2
    j0 = int(np.argmin(np.abs(D)))                 # nearest approach
    j_lo = max(0, j0-window); j_hi = min(len(K), j0+window+1)
    x = K[j_lo:j_hi]; y = D[j_lo:j_hi]
    if len(x) < 3:
        return float(K[j0])
    # fit y ≈ a x^2 + b x + c, root of quadratic closest to x[j0]
    A = np.vstack([x*x, x, np.ones_like(x)]).T
    try:
        a,b,c = np.linalg.lstsq(A, y, rcond=None)[0]
        disc = b*b - 4*a*c
        if a==0 or disc<0:
            return float(K[j0])
        r1 = (-b - math.sqrt(disc))/(2*a)
        r2 = (-b + math.sqrt(disc))/(2*a)
        # choose root within [x.min,x.max] closest to x[j0]
        candidates = [r for r in (r1,r2) if x.min()-1e-9 <= r <= x.max()+1e-9]
        return float(min(candidates, key=lambda r: abs(r - K[j0]))) if candidates else float(K[j0])
    except Exception:
        return float(K[j0])

U = {N: binder_U(data[N]["R2"], data[N]["R4"]) for N in Ns}

pairs, Kc_pairs = [], []
for i in range(len(Ns)-1):
    N1,N2 = Ns[i], Ns[i+1]
    # already on common grid; refine parabolically
    kc = parabolic_crossing(data[N1]["K"], U[N1], U[N2], window=6)
    pairs.append((N1,N2)); Kc_pairs.append(kc)

# extrapolate Kc(∞): linear vs 1/N_mid (harmonic mean)
N_mid = np.array([2/(1/p[0] + 1/p[1]) for p in pairs], dtype=np.float64)
X = 1.0/N_mid; Y = np.array(Kc_pairs, dtype=np.float64)
if len(X)>=2:
    a,b = np.polyfit(X, Y, 1)
    Kc_inf = float(b)
else:
    Kc_inf = float(Y.mean()) if len(Y) else float('nan')

# ---------- β fit using shared Kc(∞) ----------
def fit_beta_shared(Kc_star, rows, x_lo=0.03, x_hi=0.35, r_lo=0.03, r_hi=0.8):
    Xl,Yl=[],[]
    for (K,R) in rows:
        x = K - Kc_star
        if x>=x_lo and x<=x_hi and r_lo<=R<=r_hi:
            Xl.append(math.log(x)); Yl.append(math.log(R))
    if len(Xl)>=3:
        m,b = np.polyfit(np.array(Xl), np.array(Yl), 1)
        return float(m), (m,b), np.array(Xl), np.array(Yl)
    return float('nan'), (float('nan'), float('nan')), np.array([]), np.array([])

rows_all = []
for N in Ns:
    K = data[N]["K"]; R = data[N]["R"]
    rows_all += list(zip(K,R))
beta_est, (m_fit, b_fit), Xll, Yll = fit_beta_shared(Kc_inf, rows_all)

# ---------- save ----------
# CSV per N
for N in Ns:
    K = data[N]["K"]; R = data[N]["R"]; R2=data[N]["R2"]; R4=data[N]["R4"]; UU = U[N]
    with open(RUN_DIR / f"crit_N{N}.csv","w") as f:
        f.write("K,R,R2,R4,U\n")
        for k,r,r2,r4,u in zip(K,R,R2,R4,UU):
            f.write(f"{k:.6f},{r:.6f},{r2:.6f},{r4:.6f},{u:.6f}\n")

meta = {
  "run_id": RUN_ID,
  "env": {"backend":"cupy" if HAS_CP else "numpy", "dtype":"fp64"},
  "model":{"dist":DIST,"sigma_or_gamma":SIGMA},
  "integrator":{"dt":DT,"steps":STEPS,"block":BLOCK,"tail_frac":TAIL_FRAC},
  "sizes": Ns,
  "seeds": SEEDS,
  "theory":{"Kc_theory":Kc_theory},
  "results":{
    "Kc_susc": {int(N): float(data[N]["Kc_susc"]) for N in Ns},
    "binder_pairs": [{"N1":int(a),"N2":int(b),"Kc_pair":float(k)} for (a,b),k in zip(pairs,Kc_pairs)],
    "Kc_infty": float(Kc_inf),
    "beta": float(beta_est),
    "elapsed_sim_sec": elapsed_sim
  }
}
with open(RUN_DIR/"run_meta.json","w") as f: json.dump(meta,f,indent=2)

# ---------- plots ----------
# R(K)
fig1 = plt.figure(figsize=(8,4.8), dpi=140)
for N in Ns:
    plt.plot(data[N]["K"], data[N]["R"], marker="o", label=f"N={N}")
    plt.axvline(data[N]["Kc_susc"], linestyle=":", label=f"Kc_susc(N={N})≈{data[N]['Kc_susc']:.3f}")
plt.axvline(Kc_inf, linestyle="--", label=f"Kc(∞)≈{Kc_inf:.3f}")
plt.axvline(Kc_theory, linestyle="-.", label=f"Kc_theory≈{Kc_theory:.3f}")
plt.xlabel("K"); plt.ylabel("<R> (tail mean)"); plt.title("Kuramoto near-critical (nested disorder, fp64)")
plt.legend(); plt.tight_layout(); plt.savefig(RUN_DIR/"crit_R_vs_K.png"); plt.close(fig1)

# Binder U(K)
fig2 = plt.figure(figsize=(8,4.6), dpi=140)
for N in Ns:
    plt.plot(data[N]["K"], U[N], marker=".", label=f"N={N}")
for kc in Kc_pairs:
    plt.axvline(kc, linestyle=":", label=f"Kc(pair)≈{kc:.3f}")
plt.axvline(Kc_inf, linestyle="--", label=f"Kc(∞)≈{Kc_inf:.3f}")
plt.xlabel("K"); plt.ylabel("Binder-like U"); plt.title("Binder crossings (parabolic)")
plt.legend(); plt.tight_layout(); plt.savefig(RUN_DIR/"crit_binder.png"); plt.close(fig2)

# β fit
fig3 = plt.figure(figsize=(6.2,4.6), dpi=140)
if len(Xll)>=3:
    xr = np.linspace(Xll.min(), Xll.max(), 200); yr = m_fit*xr + b_fit
    plt.plot(Xll, Yll, marker="o", linestyle="", label="windowed data")
    plt.plot(xr, yr, label=f"fit β≈{m_fit:.3f}")
    plt.xlabel("log(K - Kc(∞))"); plt.ylabel("log <R>"); plt.title("Critical scaling (shared Kc)")
    plt.legend()
else:
    plt.text(0.5,0.5,"Insufficient points for β fit",ha="center",va="center"); plt.axis("off")
plt.tight_layout(); plt.savefig(RUN_DIR/"crit_beta_fit.png"); plt.close(fig3)

print("=== CNT Critical Refinement v2 — Completed ===")
print(f"Artifacts : {RUN_DIR}")
print(f"Theory Kc : {Kc_theory:.4f} | Kc(∞)≈{Kc_inf:.4f}")
for N in Ns:
    print(f"N={N} : Kc_susc≈{data[N]['Kc_susc']:.4f}")
for (a,b),kc in zip(pairs,Kc_pairs):
    print(f"Crossing(N={a},{b}) → Kc≈{kc:.4f}")
print(f"β (shared Kc) ≈ {beta_est:.3f}")
print(f"Sim elapsed (s): {elapsed_sim:.2f}")
